In [1]:
import os

In [2]:
pwd

'/home/dephinate/ASU/TextSummarization/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/home/dephinate/ASU/TextSummarization'

Create Entity 

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



Add Methods in Configuration manager to read Configurations from config.yaml for Data Ingestion and create directories to support Data Ingestion

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            param_filepath=PARAMS_FILE_PATH) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])



    # Method to read Data Ingestion Config

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


Update the Components to download data

In [8]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.utils.common import get_size
from TextSummarizer.logging import logger

In [9]:
class DataIngestion:
    def __init__(
            self,
            config: DataIngestionConfig
            ) -> None:
        self.config = config

    # Method to download the file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
                )
            logger.info(f'{filename} download! with the following info: \n{header}')
        else:
            logger.info(f"File already exits of size: {get_size(Path(self.config.local_data_file))}")

    # Method to Extract the downloaded file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

Update the Pipeline

In [11]:
with zipfile.ZipFile(Path('artifacts/data_ingestion/data.zip'), 'r') as zip_ref:
            zip_ref.extractall(Path('artifacts/data_ingestion'))

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-01-11 15:08:20,905,INFO,common,yaml file config/config.yaml loaded successfully]
[2024-01-11 15:08:20,906,INFO,common,yaml file params.yaml loaded successfully]
[2024-01-11 15:08:20,907,INFO,common,created directory at: artifacts]
[2024-01-11 15:08:20,907,INFO,common,created directory at: artifacts/data_ingestion]
[2024-01-11 15:08:24,736,INFO,2427399911,artifacts/data_ingestion/data1.zip download! with the following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 828E:40F9:70AEE5:8F8407:65A05D80
Accept-Ranges: bytes
Date: Thu, 11 Jan 2024 22:08:21 GMT
Via: 1.1 varnish
X-Served-By: cache-lax-kwhp1940045-LAX
X-Cache: HIT
X-C